In [1]:
# ==============================================================================
# Importación de Librerías
# ==============================================================================

# Conexión y Datos de APIs (Strava):
import requests # Para hacer solicitudes HTTP a la API de Strava
import json # Para codificar y decodificar datos JSON
import os # Para interactuar con el sistema operativo (crear carpetas, etc.)

# Manejo y Análisis de Datos:
import pandas as pd # Para manejar y analizar datos en DataFrames
import datetime # Para trabajar con fechas y horas

# Visualización de Datos Geográficos:
import folium # Para crear mapas interactivos
from folium.plugins import HeatMap # Para crear mapas de calor en folium
from IPython.display import display # Para mostrar objetos (como mapas) en el notebook

# Interacción con Google Drive:
from google.colab import drive # Para montar Google Drive y guardar archivos

# Carga de variables de entorno:
from dotenv import load_dotenv # Para cargar variables de entorno desde un archivo .env

# ==============================================================================
# Montar Google Drive
# ==============================================================================
drive.mount('/content/drive')

Mounted at /content/drive


**Enlace para Autorizar la conexión con la API de Strava**

 ➡️[Access](https://www.strava.com/oauth/authorize?client_id=176594&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=read,activity:read_all,profile:read_all)


In [2]:
# ============================
# INFO USUARIOS
# ============================

ENV_PATH = "/content/drive/MyDrive/Estudio/4. Cloud/3. Inteligencia artificial para la ciencia de datos/Proyecto Deportes/.env"

load_dotenv(ENV_PATH)  # Cargar el archivo .env

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
CODE = os.getenv("CODE")


In [3]:
CODE

'ed505cfc6750bce0da3f32e48f2b5075e4b7cf59'

In [4]:
# ============================
# CONEXIÓN CON LA API STRAVA
# ============================
response = requests.post(
    "https://www.strava.com/oauth/token",
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "code": CODE,
        "grant_type": "authorization_code"
    },
)


# ============================
# INFORMACIÓN DEL TOKEN
# ============================

tokens = response.json()

In [5]:
# ============================
# TOKEN DE ACCESO
# ============================

ACCESS_TOKEN = tokens["access_token"]

# ============================
# CONEXIÓN A ACTIVIDADES
# ============================

headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

activities = requests.get(
    "https://www.strava.com/api/v3/athlete/activities",
    headers=headers,
    params={"per_page": 10, "page": 1}
)

# ============================
# COMPROBAR CONEXIÓN
# ============================

for act in activities.json(): # Last Activities
    print(f"{act['name']} | {act['distance']/1000:.2f} km | {act['moving_time']/60:.1f} min")


Run 44 (70.4%) | 5.07 km | 29.7 min
Run 43 (68.4%) | 3.30 km | 21.5 min
Quiba (58.5%) | 45.37 km | 198.1 min
Run 42 (67.1%) | 5.21 km | 30.5 min
Run 41 (65%) | 5.10 km | 30.9 min
Sibateando (56.7%) | 63.36 km | 217.6 min
Run 40 (63%) | 5.01 km | 26.9 min
Romeral (54.2%) | 60.70 km | 216.2 min
Run 39 (61%) | 5.01 km | 31.8 min
El Túnel (51.8%) | 35.61 km | 119.1 min


In [6]:
# ============================
# CONFIGURACIÓN
# ============================

# Carpeta para guardar datasets
OUTPUT_FOLDER = "/content/drive/MyDrive/Estudio/4. Cloud/3. Inteligencia artificial para la ciencia de datos/Proyecto Deportes/Datasets"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ============================
# 1. Obtener lista de actividades
# ============================

def get_activities_from_date(start_date):
    """
    Obtiene una lista de actividades de Strava desde una fecha específica.

    Args:
        start_date (str): La fecha de inicio en formato "YYYY-MM-DD".

    Returns:
        list: Una lista de diccionarios, donde cada diccionario representa una actividad.
    """
    after_ts = int(datetime.datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    page = 1
    all_activities = []

    while True:
        resp = requests.get(
            "https://www.strava.com/api/v3/athlete/activities",
            headers=headers,
            params={"after": after_ts, "per_page": 200, "page": page}
        )
        data = resp.json()
        if not data or len(data) == 0:
            break
        all_activities.extend(data)
        page += 1

    return all_activities

# ============================
# 2. Obtener detalles de una actividad
# ============================
def get_activity_details(activity_id):
    """
    Obtiene los detalles completos de una actividad específica de Strava.

    Args:
        activity_id (int): El ID de la actividad de Strava.

    Returns:
        dict: Un diccionario con los detalles de la actividad. Retorna un diccionario vacío si falla.
    """
    resp = requests.get(f"https://www.strava.com/api/v3/activities/{activity_id}", headers=headers)
    if resp.status_code == 200:
        return resp.json()
    else:
      print(f"Error al obtener detalles para actividad {activity_id}: {resp.status_code}")
      return {}


# ============================
# 3. Obtener lat/lon de una actividad
# ============================
def get_latlng_stream(activity_id):
    """
    Obtiene el stream de coordenadas de latitud y longitud (latlng) para una actividad específica de Strava.

    Args:
        activity_id (int): El ID de la actividad de Strava.

    Returns:
        list: Una lista de listas, donde cada lista interior contiene [latitud, longitud].
              Retorna una lista vacía si la solicitud falla o no hay datos latlng.
    """
    resp = requests.get(
        f"https://www.strava.com/api/v3/activities/{activity_id}/streams",
        headers=headers,
        params={"keys": "latlng", "key_by_type": "true"}
    )
    if resp.status_code == 200:
        data = resp.json()
        return data.get("latlng", {}).get("data", [])
    else:
      print(f"Error al obtener latlng para actividad {activity_id}: {resp.status_code}")
      return []


# ============================
# 4. Construir datasets por deporte
# ============================
def build_datasets(activities):
    """
    Construye DataFrames de pandas separados por tipo de deporte a partir de una lista de actividades.

    Args:
        activities (list): Una lista de diccionarios, donde cada diccionario representa una actividad de Strava.

    Returns:
        dict: Un diccionario donde las claves son los nombres de los DataFrames (ej. "DF_bike")
              y los valores son los DataFrames de pandas correspondientes.
    """
    datasets = {}

    for act in activities:
        act_id = act["id"]

        # Obtener detalles extra
        details = get_activity_details(act_id)
        latlng = get_latlng_stream(act_id)
        time.sleep(0.3)  # evitar rate limit

        # Datos comunes
        sport = act["type"].lower()
        name = act.get("name")
        date = act.get("start_date_local")
        distance = act.get("distance", 0) / 1000
        elev = act.get("total_elevation_gain", 0)
        moving_time = act.get("moving_time", 0) / 60
        calories = details.get("calories", None)
        avg_speed = act.get("average_speed", 0) * 3.6
        max_speed = act.get("max_speed", 0) * 3.6
        avg_watts = act.get("average_watts", None)
        latlng_json = json.dumps(latlng)

        row, df_name = {}, None

        # Hike
        if sport == "hike":
            row = {
                "Fecha": date,
                "Nombre Actividad": name,
                "Distancia (km)": distance,
                "Desnivel (m)": elev,
                "Tiempo en Movimiento (min)": moving_time,
                "Ritmo (min/km)": moving_time / distance if distance > 0 else None,
                "Calorias": calories,
                "LatLng_JSON": latlng_json,
            }
            df_name = "DF_hike"

        # Run
        elif sport == "run":
            row = {
                "Fecha": date,
                "Nombre Actividad": name,
                "Distancia (km)": distance,
                "Desnivel (m)": elev,
                "Tiempo en Movimiento (min)": moving_time,
                "Ritmo (min/km)": moving_time / distance if distance > 0 else None,
                "Calorias": calories,
                "LatLng_JSON": latlng_json,
            }
            df_name = "DF_run"

        # Swim
        elif sport == "swim":
            row = {
                "Fecha": date,
                "Nombre Actividad": name,
                "Distancia (m)": act.get("distance", 0),
                "Tiempo en Movimiento (min)": moving_time,
                "Ritmo (min/100m)": (moving_time * 100) / act.get("distance", 1) if act.get("distance", 0) > 0 else None,
                "Calorias": calories,
                "LatLng_JSON": latlng_json,
            }
            df_name = "DF_swim"

        # Bike
        elif sport == "ride":
            row = {
                "Fecha": date,
                "Nombre Actividad": name,
                "Distancia (km)": distance,
                "Desnivel (m)": elev,
                "Tiempo en Movimiento (min)": moving_time,
                "Velocidad Promedio (km/h)": avg_speed,
                "Velocidad Máxima (km/h)": max_speed,
                "Potencia Promedio (W)": avg_watts,
                "Calorias": calories,
                "LatLng_JSON": latlng_json,
            }
            df_name = "DF_bike"

        # Guardar fila en el dataset correspondiente
        if row and df_name:
            if df_name not in datasets:
                datasets[df_name] = []
            datasets[df_name].append(row)

    # Convertir listas a DataFrames
    for key in datasets:
        datasets[key] = pd.DataFrame(datasets[key])

    return datasets

# ============================
# 5. Guardar datasets en CSV
# ============================
def save_datasets_to_csv(datasets, folder=OUTPUT_FOLDER):
    """
    Guarda los DataFrames generados en archivos CSV dentro de una carpeta específica.

    Args:
        datasets (dict): Un diccionario de DataFrames, donde las claves son los nombres de los archivos.
        folder (str): La ruta de la carpeta donde se guardarán los archivos CSV.
    """
    for name, df in datasets.items():
        filepath = os.path.join(folder, f"{name}.csv")
        df.to_csv(filepath, index=False)
        print(f"Guardado {filepath} ({len(df)} filas)")

# ============================
# 6. Uso
# ============================
if __name__ == "__main__":
    # Traer actividades desde una fecha
    activities = get_activities_from_date("2025-01-01")

    # Crear datasets
    dfs = build_datasets(activities)

    # Guardar en CSV
    save_datasets_to_csv(dfs)

Guardado /content/drive/MyDrive/Estudio/4. Cloud/3. Inteligencia artificial para la ciencia de datos/Proyecto Deportes/Datasets/DF_bike.csv (24 filas)
Guardado /content/drive/MyDrive/Estudio/4. Cloud/3. Inteligencia artificial para la ciencia de datos/Proyecto Deportes/Datasets/DF_run.csv (45 filas)
Guardado /content/drive/MyDrive/Estudio/4. Cloud/3. Inteligencia artificial para la ciencia de datos/Proyecto Deportes/Datasets/DF_hike.csv (1 filas)


In [7]:
df_bike = pd.read_csv(OUTPUT_FOLDER + "/DF_bike.csv")
df_bike.head()

,Fecha,Nombre Actividad,Distancia (km),Desnivel (m),Tiempo en Movimiento (min),Velocidad Promedio (km/h),Velocidad Máxima (km/h),Potencia Promedio (W),Calorias,LatLng_JSON
0,2025-01-19T09:11:21Z,Alto de Rosas (2.2%),55.4181,645.0,208.833333,15.9228,89.8956,71.5,2025.0,"[[4.592978, -74.161453], [4.593014, -74.161457..."
1,2025-01-28T11:27:48Z,Simón Bolívar (3.66%),36.1786,237.0,124.833333,17.3880,30.3588,49.3,1170.0,"[[4.592422, -74.160294], [4.592422, -74.160314..."
2,2025-02-02T08:39:27Z,Túnel (5.1%),35.8076,441.0,114.466667,18.7704,61.2720,95.6,1563.0,"[[4.592389, -74.160313], [4.59239, -74.160335]..."
3,2025-02-09T06:50:37Z,Sierra Morena - Cazucá (5.93%),20.8641,511.0,100.016667,12.5172,52.2792,79.2,926.0,"[[4.59249, -74.16011], [4.592493, -74.160125],..."
4,2025-02-23T07:28:51Z,Romeral (8.55%),65.4329,1122.0,248.633333,15.7896,52.9920,75.9,2446.0,"[[4.592394, -74.16007], [4.592394, -74.160091]..."


In [8]:
df_run = pd.read_csv(OUTPUT_FOLDER + "/DF_run.csv")
df_run.tail()

,Fecha,Nombre Actividad,Distancia (km),Desnivel (m),Tiempo en Movimiento (min),Ritmo (min/km),Calorias,LatLng_JSON
40,2025-08-29T16:17:06Z,Run 40 (63%),5.0149,13.0,26.900000,5.364015,433.0,"[[4.592672, -74.160454], [4.592682, -74.160455..."
41,2025-09-10T17:54:05Z,Run 41 (65%),5.1044,8.3,30.883333,6.050336,441.0,"[[4.592663, -74.160497], [4.592664, -74.160489..."
42,2025-09-12T07:29:56Z,Run 42 (67.1%),5.2119,4.4,30.483333,5.848795,449.0,"[[4.592669, -74.161374], [4.592674, -74.161474..."
43,2025-09-17T18:07:13Z,Run 43 (68.4%),3.3038,13.1,21.466667,6.497568,289.0,"[[4.592666, -74.160603], [4.592591, -74.160608..."
44,2025-09-23T07:10:31Z,Run 44 (70.4%),5.0728,10.9,29.666667,5.848184,438.0,"[[4.592941, -74.160423], [4.592951, -74.160443..."


In [9]:
df_hike = pd.read_csv(OUTPUT_FOLDER + "/DF_hike.csv")
df_hike.tail()

,Fecha,Nombre Actividad,Distancia (km),Desnivel (m),Tiempo en Movimiento (min),Ritmo (min/km),Calorias,LatLng_JSON
0,2025-06-07T09:16:37Z,Monserrate,5.5611,476.9,127.866667,22.993053,478.0,"[[4.603599, -74.061482], [4.603601, -74.061469..."


In [14]:
# ============================
# Mostrar Todas las rutas
# ============================

# Definir qué tipo deporte se quiere mostar
df_to_show = df_bike

# Obtener el nombre del DataFrame para el nombre del archivo
df_name = [name for name, obj in globals().items() if obj is df_to_show][0]


# 2. Extraer todas las coordenadas de todas las actividades
all_coords = []

for _, row in df_to_show.iterrows():
    # Asegurarse de que LatLng_JSON no esté vacío antes de intentar cargar
    if row["LatLng_JSON"]:
        coords = json.loads(row["LatLng_JSON"])
        all_coords.extend(coords)

print(f"Total de puntos cargados: {len(all_coords)}")

# 3. Crear mapa centrado en el primer punto válido
if all_coords:
    # Encontrar el primer punto válido para centrar el mapa
    valid_start_point = None
    for lat, lon in all_coords:
        if lat is not None and lon is not None:
            valid_start_point = [lat, lon]
            break

    if valid_start_point:
        m = folium.Map(location=valid_start_point, zoom_start=12)

        # 4. Añadir capa de calor
        HeatMap(all_coords, radius=6, blur=8, max_zoom=13).add_to(m)

        # 5. Mostrar en notebook
        display(m)

        # 6. Guardar en HTML en la carpeta de salida
        heatmap_filepath = os.path.join(OUTPUT_FOLDER, f"mapa_calor_{df_name.lower()}.html")
        m.save(heatmap_filepath)
        print(f"Mapa de calor guardado en {heatmap_filepath}")
    else:
        print("No se encontraron puntos válidos para centrar el mapa.")
else:
    print("No hay coordenadas para mostrar.")

Total de puntos cargados: 51567


Mapa de calor guardado en /content/drive/MyDrive/Estudio/4. Cloud/3. Inteligencia artificial para la ciencia de datos/Proyecto Deportes/Datasets/mapa_calor_df_bike.html
